##### ONS Charts

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## Imports

Standard python libraries plus determination of projdir, basic printable class, etc

In [1]:
import os
from datetime import date, datetime, timedelta
import calendar

import unittest

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

import common_core
import ons_core

## NumPy Helper Functions

Useful functionality such as moving average or rolling sum

In [2]:
def shiftRegistrations(data):
    """Shift registration data left by half a period"""
    
    # Final value is invalid (so not included in the convolution result) and needs to be zero
    result = np.append(np.convolve(data, np.array([0.5, 0.5]), mode="valid"), 0)
    
    return result

In [3]:
class TestShiftRegistrations(unittest.TestCase):
    '''Class to test rollingSum function'''   

    def testShift(self):
        '''Test processing of a list shorter than the window size'''

        actual = shiftRegistrations(np.arange(6))
        expected = np.array([0.5, 1.5, 2.5, 3.5, 4.5, 0])

        self.assertEqual((actual == expected).all(), True)

In [4]:
def getEstimatedOccurrences(cache, verbose=common_core.verbose):
    """Calculate missing values in cache"""

    estimates = {}
    
    # Use the ONS occurrences for England and Wales
    masterArea = common_core.ENGLAND_WALES

    # Non-COVID registrations for the master area, shifted left by ~3.5 days
    knownRegistrations = shiftRegistrations(cache[masterArea][ons_core.TOTAL_REGISTRATIONS] -
                                            cache[masterArea][ons_core.COVID_REGISTRATIONS])

    # Non-COVID occurrences for the master area without any shift
    knownOccurrences =  cache[masterArea][ons_core.TOTAL_OCCURRENCES] - cache[masterArea][ons_core.COVID_OCCURRENCES]

    # Run this process for all regions and nations other than the master area
    for areaName in common_core.regionNames + common_core.nationNames:
        if areaName != masterArea and areaName in cache:
            
            # Shift registrations left by half a week
            shiftedRegistrations = shiftRegistrations(cache[areaName][ons_core.TOTAL_REGISTRATIONS] -
                                                     cache[areaName][ons_core.COVID_REGISTRATIONS])

            # Estimate occurrences using a simple percentage of the known occurrences
            estimatedOccurrences = knownOccurrences * np.divide(shiftedRegistrations, knownRegistrations,
                                      out=np.zeros_like(shiftedRegistrations), where=knownRegistrations != 0) + \
                                    cache[areaName][ons_core.COVID_OCCURRENCES]
            
            # Locate the last week in the cache where total_occurrences is populated
            cacheIdx = np.where(cache[areaName][ons_core.TOTAL_OCCURRENCES] > 0)[0][-1]
            
            # Patch subsequent data using the estimates
            cache[areaName][cacheIdx + 1:][ons_core.TOTAL_OCCURRENCES] = estimatedOccurrences[cacheIdx + 1:]
            
            # Maintain a cache of estimates for testing purpopses
            estimates[areaName] = estimatedOccurrences
            
    return estimates

In [5]:
def calculateErrors(cache, estimates, verbose=common_core.verbose):
    """Calculate estimation errors"""

    masterArea = common_core.ENGLAND_WALES

    for areaName in common_core.regionNames + common_core.nationNames:
        if areaName in cache and areaName != masterArea:
            print(f"{areaName}:")
            
            totalPctMAE = 0
        
            years = range(2010, 2018)
            
            for year in years:
                startDate = f"{year}-01-01"
                stopDate = f"{year}-12-31"

                areaData = cache[areaName]

                startIdx = np.where(areaData[ons_core.WEEK_ENDED] >= startDate)[0][0]
                stopIdx = np.where(areaData[ons_core.WEEK_ENDED] < stopDate)[0][-1]

                pctMAE = 100 * np.average(np.abs(areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES] -
                                            areaData[startIdx:stopIdx][ons_core.TOTAL_REGISTRATIONS].astype(np.float64)) /
                                        areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES])

                pctMAE = 100 * np.average(np.abs(areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES] -
                                            shiftRegistrations(areaData[startIdx:stopIdx][ons_core.TOTAL_REGISTRATIONS])) /
                                        areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES])

                pctMAE = 100 * np.average(np.abs(areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES] -
                                            estimates[areaName][startIdx:stopIdx]) /
                                        areaData[startIdx:stopIdx][ons_core.TOTAL_OCCURRENCES])

                totalPctMAE += pctMAE
                
                print(f"{year} = {pctMAE:.2f}%")

            print(f"Avg  = {totalPctMAE / len(years):.2f}%")
            print()

In [6]:
def calculateAverages(cache, areaName, avgYears, verbose=common_core.verbose):
    """Calculate X year averages and ranges"""

    tmpMin = np.array([], dtype="u4")
    tmpMax = np.array([], dtype="u4")
    tmpAvg = np.array([])

    yearIdx = np.where(cache[areaName][ons_core.WEEK_NUMBER] == 1)[0]

    for i in range(avgYears, len(yearIdx)):

        if i == len(yearIdx) - 1:
            numWeeks = len(cache[areaName]) - yearIdx[i]
        else:
            numWeeks = yearIdx[i + 1] - yearIdx[i]

        grid = cache[areaName][yearIdx[i - 5]:yearIdx[i - 5] + numWeeks][ons_core.TOTAL_OCCURRENCES]
        for j in range(avgYears - 1, 0, -1):
            grid = np.vstack([
                grid,
                cache[areaName][yearIdx[i - j]:yearIdx[i - j] + numWeeks][ons_core.TOTAL_OCCURRENCES]
            ])

        tmpMin = np.append(tmpMin, np.min(grid, axis = 0))
        tmpMax = np.append(tmpMax, np.max(grid, axis = 0))
        tmpAvg = np.append(tmpAvg, np.average(grid, axis = 0))

    minimums = np.concatenate((np.zeros(yearIdx[avgYears], dtype="u4"), tmpMin))
    maximums = np.concatenate((np.zeros(yearIdx[avgYears], dtype="u4"), tmpMax))
    averages = np.concatenate((np.zeros(yearIdx[avgYears]), tmpAvg))

    return minimums, maximums, averages

## Plot Data

Simple plots of ONS data

In [7]:
SOURCE_ONS = "Source: Office for National Statistics"

GITHUB_URL = "https://logiqx.github.io/covid-stats/weekly-deaths"

In [8]:
def updateAxisLabels(ax, xTickLabels, xMin, xMax, yMin, yMax):
    '''Update axis limits and labels'''

    # Determine x-axis tick interval
    tickInterval = len(xTickLabels) // 52
    if tickInterval == 0:
        tickInterval = 1

    # Switch to an interval of 13 or 26 weeks over long time periods 
    if tickInterval > 8:
        tickInterval = 26
    elif tickInterval > 4:
        tickInterval = 13

    # Update x-axis to use desired interval
    ax.set_xlim(xmin=xMin - (xMin - ax.get_xlim()[0]) / 5, xmax=xMax + (ax.get_xlim()[1] - xMax) / 5)
    ax.set_xticks(np.arange(0, len(xTickLabels), tickInterval))
    
    # Change the x-axis to shown the actual dates
    ax.set_xticklabels(xTickLabels[::tickInterval], rotation=90)

    # Determine y-axis tick interval
    yMax *= 1.1
    if yMax > 10000:
        tickInterval = yMax // 10000 * 1000
    elif yMax > 1000:
        tickInterval = yMax // 1000 * 100
    else:
        tickInterval = yMax // 10

    # Update y-axis to use desired interval
    ax.set_ylim(ymin=0, ymax=yMax)
    ax.set_yticks(np.arange(0, yMax, tickInterval))
    
    # Ensure thousands are shown using commas
    ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Assume all charts show weekly deaths
    ax.set_ylabel('Number of weekly deaths')

    
def addTextLegend(ax, legendLoc="upper left"):
    """Add text boxes and legend to chart"""
    
    textStr = f'Created {datetime.now().strftime("%-d %b %y")}\n@Mike_aka_Logiqx'

    if legendLoc == "upper left":
        ax.text(0.01, 0.04, textStr, transform=ax.transAxes,
                horizontalalignment='left', verticalalignment='bottom', fontsize="small")
    else:
        ax.text(0.01, 0.98, textStr, transform=ax.transAxes,
                horizontalalignment='left', verticalalignment='top', fontsize="small")

    ax.legend(loc=legendLoc, borderaxespad=1, fontsize="small")

In [9]:
def plotHistory(cache, estimates, minimums, maximums, averages, avgYears, areaName, ax, verbose=common_core.verbose):
    '''Plot data for visual inspection'''
    
    areaData = cache[areaName]

    startDate = "2010-01-08"
    startIdx = np.where(areaData[ons_core.WEEK_ENDED] == startDate)[0][0]
    stopIdx = np.where(areaData[ons_core.TOTAL_REGISTRATIONS] > 0)[0][-1]

    # Total occurrences
    y_points = areaData[ons_core.TOTAL_OCCURRENCES][startIdx:stopIdx]
    x_points = np.arange(len(y_points))       
    xMin, xMax = min(x_points), max(x_points)
    yMin, yMax = min(y_points), max(y_points)
    ax.plot(x_points, y_points, label = "Total deaths - all causes", color="navy", zorder=4)

    # Average of past X years
    y_points = averages[startIdx:stopIdx]
    x_points = np.arange(len(y_points))       
    ax.plot(x_points, y_points, label = f"{avgYears} year average/mean", color="navy", linestyle="dotted")

    # Max + min of past X years
    y1_points = minimums[startIdx:stopIdx]
    y2_points = maximums[startIdx:stopIdx]
    x_points = np.arange(len(y1_points))       
    ax.fill_between(x_points, y1_points, y2_points, label = f"{avgYears} year max/min/range", color="lavender")

    # Chart titles and labels
    ax.set_title(f"Weekly Deaths in {areaName} since January 2010")

    # Add note about date of occurrence
    lastDate = datetime.strptime(areaData[ons_core.WEEK_ENDED][stopIdx - 1], "%Y-%m-%d").strftime("%A %-d %B %Y")
    textStr = f'All weekly figures are based on date of occurrence\n\n' + \
                f'Showing deaths occurring up to {lastDate}'
    ax.text(0.5, 0.98, textStr, transform=ax.transAxes, horizontalalignment='center', verticalalignment='top')

    # Add note about dates on axis
    textStr = 'Note: Date labels on the x-axis are 26 weeks apart'
    ax.text(0.5, 0.04, textStr, transform=ax.transAxes, horizontalalignment='center', verticalalignment='bottom')

    # Source can go in a different place, depending on the chart
    ax.text(0.99, 0.04, SOURCE_ONS, transform=ax.transAxes,
            horizontalalignment='right', verticalalignment='bottom', fontsize="small")

    # Update the x-axis and y-axis
    xTickLabels = []
    for weekEnded in areaData[ons_core.WEEK_ENDED][startIdx:stopIdx]:
        formattedDate = datetime.strptime(weekEnded, "%Y-%m-%d").strftime("%-d %b %y")
        xTickLabels.append(formattedDate)
    updateAxisLabels(ax, xTickLabels, xMin, xMax, yMin, yMax)

    # Add text and legend
    addTextLegend(ax)

In [10]:
def plotLatest(cache, covid, estimates, minimums, maximums, averages, avgYears, areaName, ax, verbose=common_core.verbose):
    '''Plot data for visual inspection'''
    
    areaData = cache[areaName]

    startDate = "2020-01-03"
    startIdx = np.where(areaData[ons_core.WEEK_ENDED] == startDate)[0][0]
    stopIdx = np.where(areaData[ons_core.TOTAL_REGISTRATIONS] > 0)[0][-1]

    covidStartIdx = np.where(covid[ons_core.WEEK_ENDED] == startDate)[0][0]
    covidStopIdx = np.where(covid[ons_core.COVID_REGISTRATIONS] > 0)[0][-1] + 1
    
    # Determine ratios where COVID-19 is the underlying cause
    covidUnderlying = covid[covidStartIdx:covidStopIdx][ons_core.COVID_UNDERLYING].astype(np.float64)
    covidRegistrations = covid[covidStartIdx:covidStopIdx][ons_core.COVID_REGISTRATIONS].astype(np.float64)
    underlying = np.divide(covidUnderlying, covidRegistrations,
                           out=np.zeros_like(covidUnderlying), where=covidRegistrations != 0)

    xMin = yMin = stopIdx
    xMax = yMax = 0

    # Total occurrences
    y_points = areaData[ons_core.TOTAL_OCCURRENCES][startIdx:stopIdx]
    x_points = np.arange(len(y_points))       
    xMin, xMax = min(x_points), max(x_points)
    yMin, yMax = min(y_points), max(y_points)
    if y_points[0] > yMax * 0.6:
        legendLoc = "lower left"
    else:
        legendLoc = "upper left"
    ax.plot(x_points, y_points, label = "Total deaths - all causes", color="navy", zorder=4)

    # COVID-19 underlying
    y_points = areaData[ons_core.COVID_OCCURRENCES][startIdx:stopIdx] * underlying
    x_points = np.arange(len(y_points))       
    ax.plot(x_points, y_points, label = r"COVID-19 underlying cause $^{1}$", color="red", zorder=3)

    # COVID-19 mentioned
    y_points = areaData[ons_core.COVID_OCCURRENCES][startIdx:stopIdx]
    x_points = np.arange(len(y_points))       
    ax.plot(x_points, y_points, label = r"COVID-19 mentioned on cert $^{2}$", color="darkorange", zorder=2)

    # COVID-19 not underlying
    y_points = areaData[ons_core.TOTAL_OCCURRENCES][startIdx:stopIdx] - \
                areaData[ons_core.COVID_OCCURRENCES][startIdx:stopIdx] * underlying
    x_points = np.arange(len(y_points))       
    ax.plot(x_points, y_points, label = "COVID-19 not underlying", color="royalblue", zorder=3)

    # COVID-19 not mentioned
    y_points = areaData[ons_core.TOTAL_OCCURRENCES][startIdx:stopIdx] - \
                areaData[ons_core.COVID_OCCURRENCES][startIdx:stopIdx]
    x_points = np.arange(len(y_points))       
    ax.plot(x_points, y_points, label = "COVID-19 not mentioned", color="deepskyblue", zorder=2)

    # Average of past X years
    y_points = averages[startIdx:stopIdx]
    x_points = np.arange(len(y_points))       
    ax.plot(x_points, y_points, label = f"{avgYears} year average/mean", color="navy", linestyle="dotted")

    # Max + min of past X years
    y1_points = minimums[startIdx:stopIdx]
    y2_points = maximums[startIdx:stopIdx]
    x_points = np.arange(len(y1_points))       
    ax.fill_between(x_points, y1_points, y2_points, label = f"{avgYears} year max/min/range", color="lavender")

    # Chart titles and labels
    ax.set_title(f"Weekly Deaths in {areaName} since January 2020")

    # Add note about date of occurrence
    lastDate = datetime.strptime(areaData[ons_core.WEEK_ENDED][stopIdx - 1], "%Y-%m-%d").strftime("%A %-d %B %Y")
    textStr = f'All weekly figures are based on date of occurrence\n\nShowing deaths occurring up to {lastDate}'
    ax.text(0.5, 0.98, textStr, transform=ax.transAxes, horizontalalignment='center', verticalalignment='top')

    # Source can go in a different place, depending on the chart
    ax.text(0.99, 0.98, SOURCE_ONS, transform=ax.transAxes,
            horizontalalignment='right', verticalalignment='top', fontsize="small")

    # Update the x-axis and y-axis
    xTickLabels = []
    for weekEnded in areaData[ons_core.WEEK_ENDED][startIdx:stopIdx]:
        formattedDate = datetime.strptime(weekEnded, "%Y-%m-%d").strftime("%-d %b %y")
        xTickLabels.append(formattedDate)
    updateAxisLabels(ax, xTickLabels, xMin, xMax, yMin, yMax)

    # Add text and legend
    addTextLegend(ax, legendLoc=legendLoc)

In [11]:
def plotDelays(cache, areaName, ax, verbose=common_core.verbose):
    '''Plot data for visual inspection'''
    
    fields = [
        {
            "name": ons_core.TOTAL_OCCURRENCES,
            "label": "Total deaths (date of death)",
            "color": "navy",
            "linestyle": "solid",
            "extra" : 0
        },
        {
            "name": ons_core.TOTAL_REGISTRATIONS,
            "label": "Total deaths (date of registration)",
            "color": "cornflowerblue",
            "linestyle": "dotted",
            "extra" : 1
        },
        {
            "name": ons_core.COVID_OCCURRENCES,
            "label": r"COVID-19 mentioned (occurrences) $^{2}$",
            "color": "red",
            "linestyle": "solid",
            "extra" : 0
        },
        {
            "name": ons_core.COVID_REGISTRATIONS,
            "label": r"COVID-19 mentioned (registrations) $^{2}$",
            "color": "lightcoral",
            "linestyle": "dotted",
            "extra" : 1
        }
    ]
    
    areaData = cache[areaName]

    startDate = "2020-01-03"
    startIdx = np.where(areaData[ons_core.WEEK_ENDED] == startDate)[0][0]
    stopIdx = np.where(areaData[ons_core.TOTAL_REGISTRATIONS] > 0)[0][-1]

    xMin = yMin = stopIdx
    xMax = yMax = 0
    
    # Counts direct from the ONS
    for field in fields:
        y_points = areaData[field["name"]][startIdx:stopIdx + field["extra"]]
        x_points = np.arange(len(y_points))       
        ax.plot(x_points, y_points, label = field["label"], color=field["color"], linestyle=field["linestyle"])
        if len(x_points) > 0:
            xMin = min(xMin, min(x_points))
            yMin = min(yMin, min(y_points))
            xMax = max(xMax, max(x_points))
            yMax = max(yMax, max(y_points))

    # Chart titles and labels
    ax.set_title(f"Comparison of Date of Death vs Date of Registration in {areaName}")

    # Add note about date of occurrence
    lastDate = datetime.strptime(areaData[ons_core.WEEK_ENDED][stopIdx], "%Y-%m-%d").strftime("%A %-d %B %Y")
    textStr = f'This shows the difference between date of occurrence (date of death) and date of registration\n\n' + \
                f'Showing deaths registered up to {lastDate}'
    ax.text(0.5, 0.98, textStr, transform=ax.transAxes, horizontalalignment='center', verticalalignment='top')

    # Source can go in a different place, depending on the chart
    ax.text(0.99, 0.98, SOURCE_ONS, transform=ax.transAxes,
            horizontalalignment='right', verticalalignment='top', fontsize="small")

    # Update the x-axis and y-axis
    xTickLabels = []
    for weekEnded in areaData[ons_core.WEEK_ENDED][startIdx:stopIdx + 1]:
        formattedDate = datetime.strptime(weekEnded, "%Y-%m-%d").strftime("%-d %b %y")
        xTickLabels.append(formattedDate)
    updateAxisLabels(ax, xTickLabels, xMin, xMax, yMin, yMax)

    # Add text and legend
    addTextLegend(ax)

In [12]:
def plotPolar(cache, estimates, areaName, fig, verbose=common_core.verbose):
    '''Plot data as polar plot - aka Florence Nightale diagram'''

    # Purely for convenience
    areaData = cache[areaName]

    # Determine date range
    startDate = "2001-01-05"
    startIdx = np.where(areaData[ons_core.WEEK_ENDED] == startDate)[0][0]
    stopIdx = np.where(areaData[ons_core.TOTAL_REGISTRATIONS] > 0)[0][-1]

    # Data for the specific date range
    occurrences = areaData[ons_core.TOTAL_OCCURRENCES][startIdx:stopIdx]
    weekEndeds = areaData[ons_core.WEEK_ENDED][startIdx:stopIdx]
    weekNumbers = areaData[ons_core.WEEK_NUMBER][startIdx:stopIdx]
    weekOffsets = np.where(weekNumbers == 1)[0]

    # Pre-claculate theta values for polar plot
    thetas = []
    currYear = None
    yDays = 365
    for weekEnded in weekEndeds:
        if weekEnded[:4] != currYear:
            currYear = int(weekEnded[:4])
            yDays = 365
            if calendar.isleap(currYear):
                yDays += 1
                
        yDay = datetime.strptime(weekEnded, "%Y-%m-%d").timetuple().tm_yday - 1
        thetas.append(2 * np.pi * yDay / yDays)

    # Keep track of maximum y-value
    yMax = 0
    
    # Create subplot and attach to figure
    ax = plt.subplot(111, projection='polar')
    fig.add_subplot(ax)

    # Choose colour cycler
    ax.set_prop_cycle(plt.cycler("color", plt.cm.viridis(np.linspace(1, 0, len(weekOffsets)))))

    # Plot the years individually but connecting them up
    for i in range(len(weekOffsets)):
        year = int(weekEndeds[weekOffsets[i]][:4])

        if i == len(weekOffsets) - 1:
            y_points = occurrences[weekOffsets[-1] - 1:]
            theta = thetas[weekOffsets[-1] - 1:]
        elif i > 0:
            y_points = occurrences[weekOffsets[i] - 1:weekOffsets[i + 1]]
            theta = thetas[weekOffsets[i] - 1:weekOffsets[i + 1]]
        else:
            y_points = occurrences[weekOffsets[i]:weekOffsets[i + 1] + 1]
            theta = thetas[weekOffsets[i]:weekOffsets[i + 1] + 1]
            
        # Update y-max
        yMax = max(yMax, max(y_points))

        # Plot the data
        ax.plot(theta, y_points, label=year)
   
    # Consider 1 Jan to be "north" and move around the plot clockwise
    ax.set_theta_zero_location("N")
    ax.set_theta_direction(-1)

    # Calculate desired interval of radial ticks
    for i in [10000, 1000, 100]:
        if yMax > i:
            tickInterval = (yMax // i) * i / 10
            if yMax % tickInterval < tickInterval / 2:
                yMax = (yMax // tickInterval + 1) * tickInterval
            else:
                yMax = (yMax // tickInterval + 2) * tickInterval
            break

    # Update y-axis to use desired interval
    ax.set_rmin(0)
    ax.set_rmax(yMax)
    ax.set_rticks(np.arange(0, yMax, tickInterval))
    ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))
    
    # Angle radial labels towards NWbN / 11 o'clock / 1 Nov
    ax.set_rlabel_position(-30)  
    #ax.grid(True)

    # Set theta labels to the 1st of each month
    angles = []
    labels = []
    for i in range(1, 13):
        dt = date(date.today().year, i, 1)
        angles.append((dt.timetuple().tm_yday - 1) / 365.4 * 360)
        labels.append(dt.strftime("1 %b"))
    ax.set_thetagrids(angles, labels=labels)

    # Add note about date of occurrence
    textStr = 'All weekly figures are based on date of occurrence'
    fig.text(0.5, 0.95, textStr, horizontalalignment='center', verticalalignment='top')
    
    lastDate = datetime.strptime(areaData[ons_core.WEEK_ENDED][stopIdx - 1], "%Y-%m-%d").strftime("%A %-d %B %Y")
    textStr = f'Showing deaths occurring up to {lastDate}'
    fig.text(0.5, 0.93, textStr, horizontalalignment='center', verticalalignment='top')

    # Source can go in a different place, depending on the chart
    fig.text(0.96, 0.98, SOURCE_ONS, horizontalalignment='right', verticalalignment='top', fontsize="small")

    # Add text and legend
    textStr = f'Created {datetime.now().strftime("%-d %b %y")} by @Mike_aka_Logiqx'
    fig.text(0.96, 0.10, textStr, horizontalalignment='right', verticalalignment='bottom', fontsize="small")

    ax.legend(loc='upper left', fontsize="small", bbox_to_anchor=(1, 1.1))

In [13]:
def legacy(cache):
    """Date ranges for some legacy testing"""
    
    # Check heatwave of August 2003 - https://www.eurosurveillance.org/content/10.2807/esm.10.07.00558-en
    startDate = '2003-07-01'
    stopDate = '2003-09-01'

    # Show reg delays
    startDate = date(2010, 1, 1).strftime('%Y-%m-%d')
    stopDate = date(2020, 1, 8)

    # Same range as error check
    startDate = '2014-01-01'
    stopDate = '2018-12-31'


def plotAreas(reportErrors = False, verbose=common_core.verbose):
    '''Plot data for visual inspection'''

    # Load all of the data from disk
    cache = ons_core.loadCsvFiles(ons_core.ONS_DEATHS, "weekly", verbose = False)
    covid = ons_core.loadCsvFile(ons_core.ONS_DEATHS, "weekly", "nation", "england_wales_covid", verbose=common_core.verbose)
    
    # Estimate the number of occurrences where necessary
    estimates = getEstimatedOccurrences(cache, verbose=verbose)
   
    # Shift COVID-19 registrations left so they can be used for occurrences
    covid[ons_core.COVID_REGISTRATIONS] = shiftRegistrations(covid[ons_core.COVID_REGISTRATIONS])
    covid[ons_core.COVID_UNDERLYING] = shiftRegistrations(covid[ons_core.COVID_UNDERLYING])

    # Evaluation of estimation errors
    if reportErrors:
        calculateErrors(cache, estimates, verbose=verbose)

    # Figures for each area
    for areaName in common_core.nationNames + common_core.regionNames:
        if areaName in cache:
            
            fig, axs = plt.subplots(3, figsize=(16, 18), dpi=100)

            avgYears=5
            minimums, maximums, averages = calculateAverages(cache, areaName, avgYears, verbose=verbose)

            plotHistory(cache, estimates, minimums, maximums, averages, avgYears, areaName, axs[0], verbose=verbose)
            plotLatest(cache, covid, estimates, minimums, maximums, averages, avgYears, areaName, axs[1], verbose=verbose)
            plotDelays(cache, areaName, axs[2], verbose=verbose)

            ucodStr = r"$^{1}$ The underlying cause of death is defined as the disease or injury which initiated " + \
                       "the chain of morbid events leading directly to death."
            mentionStr = r"$^{2}$ COVID-19 may be mentioned in part I (conditions leading to death) or part II " + \
                          "(significant conditions contributing to death) of the medical certificate cause of death (MCCD)."

            gitHubStr = "\nA detailed explanation along with a link to the code and data " + \
                         f"that were used to create these charts can be found at {GITHUB_URL}"
            
            textStr = "\n".join([ucodStr, mentionStr, gitHubStr])
            fig.text(0.5, 0.02, textStr, horizontalalignment='center', verticalalignment='bottom')

            plt.subplots_adjust(hspace=0.4)
            plt.show()

            partName = os.path.join("docs", "weekly-deaths", common_core.getSafeName(areaName) + ".png")
            fileName = os.path.join(common_core.projdir, partName)

            print(f"Saving {partName}...")
            fig.savefig(fileName, bbox_inches='tight', facecolor='w')
            plt.close(fig)
            
    # Figures for multiple areas (combined) - 5 and 10 year averages
    areaGroups = \
    [
        ["London", "South East", "East of England"],
        ["North West", "North East", "Yorkshire and The Humber"],
        ["West Midlands", "East Midlands", "South West"],
        ["England", "Wales", "England and Wales"]
    ]
    
    groupNo = 1
    for areaNames in areaGroups:
        for avgYears in (5, 10):

            fig, axs = plt.subplots(len(areaNames), figsize=(16, 6 * len(areaNames)), dpi=100)

            axsIdx = 0
            for areaName in areaNames:
                minimums, maximums, averages = calculateAverages(cache, areaName, avgYears, verbose=verbose)
                plotLatest(cache, covid, estimates, minimums, maximums, averages, avgYears, areaName, axs[axsIdx], verbose=verbose)
                axsIdx += 1

            ucodStr = r"$^{1}$ The underlying cause of death is defined as the disease or injury which initiated " + \
                       "the chain of morbid events leading directly to death."
            mentionStr = r"$^{2}$ COVID-19 may be mentioned in part I (conditions leading to death) or part II " + \
                          "(significant conditions contributing to death) of the medical certificate cause of death (MCCD)."

            gitHubStr = "\nA detailed explanation along with a link to the code and data " + \
                         f"that were used to create these charts can be found at {GITHUB_URL}"

            textStr = "\n".join([ucodStr, mentionStr, gitHubStr])
            fig.text(0.5, 0.02, textStr, horizontalalignment='center', verticalalignment='bottom')

            plt.subplots_adjust(hspace=0.4)
            plt.show()

            partName = os.path.join("docs", "weekly-deaths", f"{avgYears}_years_{groupNo}" + ".png")
            fileName = os.path.join(common_core.projdir, partName)

            print(f"Saving {partName}...")
            fig.savefig(fileName, bbox_inches='tight', facecolor='w')
            plt.close(fig)

        groupNo += 1
            
    # Florence Nightingale diagrams for each year
    for areaName in common_core.nationNames + common_core.regionNames:
        if areaName in cache:
            
            fig = plt.figure(figsize=(12, 12), dpi=100)
           
            plotPolar(cache, estimates, areaName, fig, verbose=verbose)

            fig.suptitle(f"Weekly Deaths in {areaName} since January 2001", fontsize="x-large", fontweight="bold")

            gitHubStr = f"A detailed explanation along with a link to the code and data can be found at {GITHUB_URL}"
            
            fig.text(0.5, 0.08, gitHubStr, horizontalalignment='center', verticalalignment='top')

            plt.subplots_adjust(hspace=0.4)
            fig.show()

            partName = os.path.join("docs", "weekly-deaths", common_core.getSafeName(areaName) + "_polar.png")
            fileName = os.path.join(common_core.projdir, partName)

            print(f"Saving {partName}...")
            fig.savefig(fileName, bbox_inches='tight', facecolor='w')
            plt.close(fig)

In [14]:
def saveIndexPage(h2_text, suffix=""):
    """Save HTML page as index to images"""

    indexPageDetails = \
    {
        "TITLE_TEXT": "Weekly Deaths",
        "H1_TEXT": "Weekly Deaths in England and Wales",
        "H2_TEXT": h2_text,
        "CUSTOM_CSS": "css/custom.css",
        "FURTHER_HREF": "."
    }
    fileName = os.path.join(common_core.projdir, "docs", "weekly-deaths", "regions" + suffix + ".html")
    indexPage = common_core.IndexPage(fileName, indexPageDetails)

    nationNames = [common_core.ENGLAND, common_core.WALES, common_core.ENGLAND_WALES]

    galleryDetails = \
    {
        "P_TEXT": ", ".join(nationNames)
    }
    gallery = indexPage.addGallery(galleryDetails["P_TEXT"], galleryDetails)

    for nationName in nationNames:
        fileName = os.path.join(common_core.projdir, "docs", "weekly-deaths",
                                common_core.getSafeName(nationName) + suffix + ".png")
        
        figureDetails = \
        {
            "ALT_TEXT": nationName,
            "CAPTION_TEXT": nationName
        }
        figure = gallery.addFigure(nationName, ".", fileName, figureDetails)
        figure.createThumb(suffix="_thumb")

    regionGroups = \
    [
        ["London", "South East", "East of England"],
        ["North West", "North East", "Yorkshire and The Humber"],
        ["East Midlands", "West Midlands", "South West"]
    ]

    for regionNames in regionGroups:
        galleryDetails = \
        {
            "P_TEXT": ", ".join(regionNames)
        }
        gallery = indexPage.addGallery(galleryDetails["P_TEXT"], galleryDetails)

        for regionName in regionNames:
            fileName = os.path.join(common_core.projdir, "docs", "weekly-deaths",
                                    common_core.getSafeName(regionName) + suffix + ".png")

            figureDetails = \
            {
                "ALT_TEXT": regionName,
                "CAPTION_TEXT": regionName
            }
            figure = gallery.addFigure(regionName, ".", fileName, figureDetails)
            figure.createThumb(suffix="_thumb")

    html = indexPage.saveHtml()

In [15]:
def saveAltIndexPage(h2_text, suffix=""):
    """Save HTML page as index to images"""

    indexPageDetails = \
    {
        "TITLE_TEXT": "Weekly Deaths",
        "H1_TEXT": "Weekly Deaths in England and Wales",
        "H2_TEXT": h2_text,
        "CUSTOM_CSS": "css/alt.css",
        "FURTHER_HREF": "."
    }
    fileName = os.path.join(common_core.projdir, "docs", "weekly-deaths", "regions" + suffix + ".html")
    indexPage = common_core.IndexPage(fileName, indexPageDetails)

    for numYears in [5, 10]:

        baseName = f"{numYears}_years"
        galleryDetails = \
        {
            "P_TEXT": f"Comparing to {numYears} Average"
        }
        gallery = indexPage.addGallery(galleryDetails["P_TEXT"], galleryDetails)

        for imgNo in range(1, 5):
            fileName = os.path.join(common_core.projdir, "docs", "weekly-deaths", f"{numYears}_years_{imgNo}.png")
        
            figureName = f"Group {imgNo}"
            figureDetails = \
            {
                "ALT_TEXT": figureName,
                "CAPTION_TEXT": figureName
            }
            figure = gallery.addFigure(figureName, ".", fileName, figureDetails)
            figure.createThumb(suffix="_thumb")

    html = indexPage.saveHtml()

## Automated Testing

In [16]:
if __name__ == '__main__':

    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


## Interactive Testing

In [17]:
if __name__ == '__main__':

    plotAreas(reportErrors = False, verbose = False)
    
    saveIndexPage("All-cause Mortality from 2010-2021")
    saveIndexPage("Radar / Polar Charts for 2001-2021", suffix="_polar")
    saveAltIndexPage("All-cause mortality during 2020-2021", suffix="_alt")